In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus'

corpus, labels = load_corpus('../data/' + file_name + '.txt')

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 268
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1449 words
filter frequent words: 605 words
filter letter words: 604 words
filter stop words: 403 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [6]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [7]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [9]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [10]:
min_samples = 7
eps = 0.32
expand_rate = 0.05
epoch = 15

onehot_corpus = get_onehot(idx_corpus, 'normal')
# onehot_corpus = get_onehot(filtered_corpus, 'tfidf')

In [11]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

136.8647991242515
[55 30 29 12 11  8 15 13  8 11 17 10 13  9 12  8  7]


#### Iterative New SDC

In [12]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

131.21642723976302
[62 32 15 23 15 17 10  8 26 14  9 12 17  8]
123.6078853341646
[71 72 15 18 15  9  9 24 11  9 15]
121.3272365420589
[71 81 15 16 15 18  8 22 10 12]
121.87904204964035
[76 77 15 16 14 16  8 24 10 12]
122.11223197229268
[76 77 15 16 14 15  8 25 10 12]


In [13]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
9 3 0.3645705286387563
3 0 0.31460425969915773
[0, 1, 2, 0, 4, 5, 6, 7, 8, 0]


### Result

In [14]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [15]:
# result.to_csv('../data/results/new_sdc/' + file_name + '.csv')

# result = pandas.read_csv('../data/results/new_sdc/' + file_name + '.csv')

print(eval_cluster(onehot_corpus, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

122.11223197229268
[76 77 15 16 14 15  8 25 10 12]


In [16]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 4, 5, 6, 7, 8), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
3 คิว เปิดช่องเยอะๆหน่อยค่ะ คนไหนใช้ช่องทางด่วน ผิดกติกา จะยอมช่วยเห็นใจคนทำถูกกติกาด้วยจ้า
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
10 ทุกคนล้วนแล้ว แต่มีหน้าที่ของเขา คนเราควรให้โอกาสซึ่งกันและกันนะครับให้คิดว่าคนดีก็มี คนไม่ดีก็มีนะครับ ผมไม่ได้โลกสวยนะครับ แต่ผมก็ทำงานในด้านบริการเหมือนกัน ถ้าไม่มายืนตรงจุดที่เขายืนคุณก็จะไม่รู้ คิดบวกนะครับ พนักงานเหมือนกันนะครับ ปรับปรุงตรงไหนแล้วดี ก็ควรที่ต้องปรับปรุงแก้ไขส่วนที่ผิดพลาดของตัวเรา ดีแล้วยังดีได้อีกนะครับ Mc.Tom
13 เค้าเตอร์มีเกือบ100ช่อง เปิดให้จ่ายตังไม่ถึง10ช่อง พี่ทำไว้โก้ๆหรอ ต่อคิวจ่ายตังกันไปดิ 10กว่านาที
14 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
16 ราคาสินค้า ตอนโปรโมชั่น บางอันหาไม่เจอ เหมือนพนักงานเอาไปซ่อน ให้ลูกค้าวิ่งไล่หา #ไอ้ซั้ซไม่ใช่มอญซ่อนผ้า ปล.รปภ บางสาขา หน้าตาดุมาก เวลาเข้าห้างนึกว่าทำไรผิดตลอดเวลา 😂
17 เทสโก้โลตัสสาขาบางกะปิทีวีตัวนี้อยู่บริเวณหน้าร้าน Treephaphan Spa

In [17]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
13 เค้าเตอร์มีเกือบ100ช่อง เปิดให้จ่ายตังไม่ถึง10ช่อง พี่ทำไว้โก้ๆหรอ ต่อคิวจ่ายตังกันไปดิ 10กว่านาที
14 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
16 ราคาสินค้า ตอนโปรโมชั่น บางอันหาไม่เจอ เหมือนพนักงานเอาไปซ่อน ให้ลูกค้าวิ่งไล่หา #ไอ้ซั้ซไม่ใช่มอญซ่อนผ้า ปล.รปภ บางสาขา หน้าตาดุมาก เวลาเข้าห้างนึกว่าทำไรผิดตลอดเวลา 😂
17 เทสโก้โลตัสสาขาบางกะปิทีวีตัวนี้อยู่บริเวณหน้าร้าน Treephaphan Spa ชั้นลอย(ชั้นเดียวกับไปรษณีย์)ใกล้ประตูออกลานจอดรถวันนั้นผมเดินชนเต็มๆ และผมเชื่อว่าต้องมีลูกค้าท่านอื่นเดินชนทีวีตัวนี้มาแล้วหลายครั้ง มันไม่ควรอยู่ตรงนี้นะครับเพราะนี่คือทางเดิน และมันก็ไม่ได้ชิดเข้ามุมกำแพงเลย เดินเลี้ยวขวาจากทีวีตัวนี้ไปก็ลงไปชั้นG ซึ่งมันเป็นทางหลักที่คนเข็นรถลงจากซุปเปอร์ฯใช้ประจำนะครับ ไม่ใช่ทางลับที่ไม่มีคนเดินผ่าน เพดานชั้นนี้ก็ค่อนข้างต่ำอยู่แล้วแถมเอาทีวีตัวนี้มาติด ต้องคนที่มีความสูง150ลงไปเท่านั้นแ

In [18]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

2 ['เลิก', 'ข้าว', 'ดี', 'อี', 'เครื่อง', 'น้ำ']
8 ['อย่า', 'บังคับ', 'น้อง', 'เชียร์', 'อย่า', 'งี่', 'เง่า', 'กี่', 'ปี']
13 ['เค้าเตอร์', 'ช่อง', 'จ่าย', 'ตัง', 'ช่อง', 'ทำ', 'หรอ', 'คิว', 'จ่าย', 'ตัง', 'นาที']
14 ['เลือก', 'ผัก', 'เลือก', 'เลือก', 'เงิน', 'พนักงาน', 'ยัด', 'ผัก', 'ใส่', 'ถุง', 'ผัก', 'หัก']
16 ['ราคา', 'สินค้า', 'ตอน', 'โปรโมชั่น', 'หา', 'เจอ', 'เหมือน', 'พนักงาน', 'ซ่อน', 'ลูกค้า', 'วิ่ง', 'ไล่', 'หา', 'ไอ้', 'ซ่อน', 'ผ้า', 'หน้าตา', 'เวลา', 'ห้าง', 'นึก', 'เวลา']
17 ['สาขา', 'กะปิ', 'ตัว', 'บริเวณ', 'หน้า', 'ร้าน', 'ชั้น', 'ชั้น', 'ไปรษณีย์', 'ประตู', 'จอด', 'รถ', 'ผม', 'เดิน', 'ชน', 'เต็ม', 'ผม', 'ลูกค้า', 'เดิน', 'ชน', 'ตัว', 'เดิน', 'มุม', 'เดิน', 'ขวา', 'ตัว', 'ชั้น', 'หลัก', 'คน', 'เข็น', 'รถ', 'ประจำ', 'คน', 'เดิน', 'ชั้น', 'ต่ำ', 'อยู่แล้ว', 'แถม', 'ตัว', 'ติด', 'คน', 'เดิน', 'ผม', 'ผม', 'ตัว', 'บริการ', 'ดี', 'สะดวก', 'ลูกค้า', 'บริการ', 'ตัว', 'สะดวก', 'แถม', 'ทำ', 'ผม', 'เจ็บ', 'ตัว', 'ผม', 'ผม', 'ฐานะ', 'ลูกค้า', 'ตัว', 'อยู่แล้ว', 'ดี', 'สะดวก', 'หวั

In [19]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['โลตัส', 'สาขา', 'จ้าง', 'พนักงาน', 'พนักงาน', 'แผนก', 'คน', 'หน้าที่', 'ตัว', 'รับผิดชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัว', 'เละ', 'โดน', 'ด่า', 'โดน', 'ด่า', 'เครียด', 'หน้า', 'ใส่', 'ลูกค้า', 'ทำ', 'งาน', 'บริการ', 'ลูกค้า', 'ดี', 'ลูกค้า', 'รัก', 'แนะนำ', 'ใจ', 'เด็ก', 'โลตัส', 'เก่า']
โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะกว่านี้​ ไม่ควรเรียกพนักงานแผนกอื่นไปลงเครื่องแคชเชียร์​บ่อยจนเกินไป​ ทุกคนต้องมีหน้าที่ที่ตัวเองต้องรับผิดชอบ​ พอช่วยแคชเชียร์เสร็จ​ งานแผนกตัวเองเละก็โดนด่า​ พอโดนด่าก็เกิดความเครียด​ หน้าบึ้งตึงใส่ลูกค้า​ ทำงานไม่มีความสุข​ บริการลูกค้าไม่ดี​ แล้วลูกค้าก็ร้องเรียน​ เพราะรักจึงอยากแนะนำ​ จากใจเด็กโลตัสเก่า​
['โลตัส', 'สาขา', 'จ้าง', 'พนักงาน', 'พนักงาน', 'แผนก', 'คน', 'หน้าที่', 'ตัว', 'รับผิดชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัว', 'เละ', 'โดน', 'ด่า', 'โดน', 'ด่า', 'เครียด', 'หน้า', 'ใส่', 'ลูกค้า', 'ทำ', 'งาน', 'บริการ', 'ลูกค้า', 'ดี', 'ลูกค้า', 'รัก', 'แนะนำ', 'ใจ', 'เด็ก', 'โลตัส', 'เก่า']
โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะ